# Part I. ETL Pipeline for Pre-Processing the Files

### 1. Import Python packages 

In [36]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

### 2. Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/home/workspace


### 3. Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
#reading the csv file and showing first 5 rows
df_log = pd.read_csv('event_datafile_new.csv')
df_log.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Muse,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Supermassive Black Hole (Twilight Soundtrack V...,42
1,Beastie Boys,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Lighten Up,42
2,Shakira,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Pienso En Ti,42
3,Selena,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Amor Prohibido,42
4,Kid Cudi Vs Crookers,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Day 'N' Nite,42


# Part II. Creating Table and Making Queries in Apache Cassandra

## 4. Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### 5. Creating Cluster and Keyspace

#### 5.1 Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance of local machine (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### 5.2 Create Keyspace

In [7]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### 5.3 Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## 6. Creating Tables and Queries

* In relational database the database schema is created first and then the queries are generated. In Apache Cassandra however, the schema is designed based on the queries we would like to run. Hence, at this point we need to model three separate tables for three queries.

### 6. 1 Query-1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### 6.1.1 Data Modelling for Query-1

In this case, our final query should give us the artist name, name and length of the song played with the sessionId = 338 and itemInSession = 4. In order to answer Query-1, we first need to create a table that contains all the artist, song, length, sessionid and iteminsession. 

Since this query is about the artist(s) with song title and length of the song, we name the table **artist_library_1**. We now need to choose the primary key based on the data we need to extract from the table. In order to obtain an appropriate result from the query- 

* We need to insert sessionId, itemInSession, artist, song and song length into the table with the appropriate data type.


* We choose **sessionId** as the **partition key** and **itemInSession** as the **clustering key**. When we retrieve the data with the query from the table, we also need to make sure we follow this order of partition key that is the - sessionId first followed by the clustering key- itemInSession in the WHERE clause.

#### 6.1.2 Creating Table for Query-1 and Inserting Data

In [9]:
## Assigning INSERT statements into the `query` variable
query = "CREATE TABLE IF NOT EXISTS artist_library_1"
query  = query  + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession)) "

try:
    session.execute(query)
    
except Exception as e:
    print(e)

In [10]:
# csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
  
    for line in csvreader:
                                                       
        ## Assigning column elements for each column in the INSERT statement.
        query = "INSERT INTO artist_library_1 (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s) " 
        
        ## Execute to insert values into the columns
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9],  float(line[5])))

#### 6.1.3 Verify data insertion into the table for Query-1

In [11]:
## Add in the SELECT statement to verify the data was entered into the table
query = 'SELECT * FROM artist_library_1 LIMIT 5'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    """ This function converts the first five rows of artist_library_1 into the dataframe artist_lib_df"""
    artist_library_df = pd.DataFrame(list(rows))

In [12]:
#check if the table has been created for query 1
artist_library_df

,sessionid,iteminsession,artist,length,song
0,23,0,Regina Spektor,191.085266,The Calculation (Album Version)
1,23,1,Octopus Project,250.957916,All Of The Champs That Ever Lived
2,23,2,Tegan And Sara,180.061584,So Jealous
3,23,3,Dragonette,153.390564,Okay Dolores
4,23,4,Lil Wayne / Eminem,229.589752,Drop The World


#### 6.1.4 Run Query-1 & Verify the result

In [16]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4       

query = "select artist, song, length from artist_library_1 WHERE sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
#convert the query_1 elements into dataframe called - query_1_df
for row in rows:
    query_1_df = pd.DataFrame(list(rows))
    

In [17]:
#show the result for query 1
query_1_df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [18]:
#checking the accuracy of the result using python's query function- should show the same result as query_1_df 
df_log.query ('sessionId == 338 and itemInSession == 4')[['artist', 'song', 'length']]

,artist,song,length
1258,Faithless,Music Matters (Mark Knight Dub),495.3073


### 6. 2 Query-2: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### 6.2.1 Data Modelling for Query-2

In this case, our final query should give us the artist name,  song sorted by itemInSession, first and last name of the user for userid = 10, and the sessionId = 338.

In order to answer Query-2, we first need to create a table that contains all the artist name, song, itemInSession, first and last name of the users. 

Since this query reflects song playlist in session, we name the table **song_playlist_session_1**.

We now need to choose the primary key based on the data we need to extract from the table. In order to obtain the appropriate result from the query- 

* We need to insert userId, sessionId, artist, itemInSession, song, firstName and lastName into the table with the appropriate data type.


* We choose **userId** and  **sessionId** as the **partition key**, **itemInSession** as the **clustering keys**. The partition keys will be used to uniquely identify each entry while the clustering key will be used for the purpose of sorting.

When we retrieve the data with the query from the table, with the WHERE clause the data will be brought for the specific userid and sessionId mentioned and then will be sorted by the itemInSession as this is the clustering key.

#### 6.2.2 Creating Table for Query-2 and Inserting Data

In [19]:
## Assigning INSERT statements into the `query` variable
query = "CREATE TABLE IF NOT EXISTS song_playlist_1"
query  = query  + "(userId int, sessionId int,  itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession) ) "

try:
    session.execute(query)
    
except Exception as e:
    print(e)

In [20]:
# csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
  
    for line in csvreader:
                                                       
        ## Assigning column elements for each column in the INSERT statement.
        query = "INSERT INTO song_playlist_1 (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) " 
        
        ## Execute to insert values into the columns
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]), line[0], line[9], line[1], line[4] ))

#### 6.2.3 Verify data insertion into the table for Query-2

In [21]:
## Add in the SELECT statement to verify the data was entered into the table
query = 'SELECT * FROM song_playlist_1 LIMIT 5'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    """ This function converts the first five rows of artist_library_1 into the dataframe artist_lib_df"""
    song_playlist_df = pd.DataFrame(list(rows))

In [22]:
song_playlist_df

,userid,sessionid,iteminsession,artist,firstname,lastname,song
0,58,768,0,System of a Down,Emily,Benson,Sad Statue
1,58,768,1,Ghostland Observatory,Emily,Benson,Stranger Lover
2,58,768,2,Evergreen Terrace,Emily,Benson,Zero
3,85,776,2,Deftones,Kinsley,Young,Head Up (LP Version)
4,85,776,3,The Notorious B.I.G.,Kinsley,Young,Playa Hater (Amended Version)


#### 6.2.4 Run Query-2 & Verify the result

In [43]:
## Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## userId = 10, and sessionid = 182        

query = "select artist, song, firstname, lastname from song_playlist_1 WHERE userId = 10 and sessionId = 182 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#use pretty table to display data in tabular form include headings
t= PrettyTable (['artist', 'song', 'firstname', 'lastname'])
    
#convert the query_2 elements into prettytable
for row in rows:
    t.add_row(row)

#print the prettytable to show the result of query-2
print(t)

    

+-------------------+------------------------------------------------------+-----------+----------+
|       artist      |                         song                         | firstname | lastname |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


In [25]:
#use python query function to varify the result of query-2, should return the same result as query_2_df
df_log.query('userId == 10 and sessionId == 182')[['artist', 'song', 'firstName', 'lastName']]

,artist,song,firstName,lastName
6054,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
6055,Three Drives,Greece 2000,Sylvie,Cruz
6056,Sebastien Tellier,Kilometer,Sylvie,Cruz
6057,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 6.3 Query-3: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### 6.3.1 Data Modelling for Query-3

In this case, our final query should give us the first and last name of the user who listened to the song- **All Hands Against His Own**.

In order to answer Query-3, we first need to create a table that contains all the songs, userid, first and last name of the users. 

Since this query reflects the users who listen to a particular song, we name the table **user_name_1**.

We now need to choose the primary key based on the data we need to extract from the table. In order to obtain an appropriate result from the query- 

* We need to insert song, userId, firstName and lastName into the table with the appropriate data type.


* We choose **song** as the **partition key**, **userId** as the **clustering key**. Together these two will uniquely identify the instances when this particular song is played. 



#### 6.3.2 Creating Table for Query-3 and Inserting Data

In [26]:
## Assign the INSERT statements into the `query` variable
query = "CREATE TABLE IF NOT EXISTS user_name_1"
query  = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"

try:
    session.execute(query)
    
except Exception as e:
    print(e)

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
  
    for line in csvreader:
                                                       
        ## Assigning column elements for each column in the INSERT statement.
        query = "INSERT INTO user_name_1 (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s) " 
        
        ## execute to insert values into the columns
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### 6.3.3 Verify data insertion into the table for Query-3

In [28]:
## Add in the SELECT statement to verify the data was entered into the user_name_1 table
query = 'SELECT * FROM user_name_1 LIMIT 5'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
for row in rows:
    """ This function converts the first five rows of user_name table into the dataframe user_name_df"""
    user_name_df = pd.DataFrame (list(rows))

In [29]:
#see if a table has been created for query 3- should show 5 rows with song, userid, firstname and lastname
user_name_df

,song,userid,firstname,lastname
0,Wonder What's Next,49,Chloe,Cuevas
1,In The Dragon's Den,49,Chloe,Cuevas
2,Too Tough (1994 Digital Remaster),44,Aleena,Kirby
3,Rio De Janeiro Blue (Album Version),49,Chloe,Cuevas
4,My Place,15,Lily,Koch


#### 6.3.4 Run Query-3 & Verify the result

In [33]:
#run the query
query = "select firstname, lastname from user_name_1 WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['firstName', 'lastName'])
    
# convert the query 3 result into a dataframe
for row in rows:
    t.add_row(row)


In [37]:
#show the result of query-3 in pretty tables
print(t)

+------------+----------+
| firstName  | lastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


In [38]:
#checking the result- should match the output of query_3_df
df_log.query ('song == "All Hands Against His Own"')[['firstName', 'lastName']]

,firstName,lastName
1804,Sara,Johnson
1935,Tegan,Levine
3541,Jacqueline,Lynch


### 7. Drop the tables before closing out the sessions

In [44]:
#drop the artist_library_1 table created for query-1
query = "drop table artist_library_1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [45]:
#drop the song_playlist_session_1 table created for query-2
query = "drop table song_playlist_session_1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [46]:
#drop the user_name_1 table created for query-2
query = "drop table user_name_1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### 8. Close the session and cluster connection¶

In [47]:
#shutdown session and cluster
session.shutdown()
cluster.shutdown()

### 9. Refereence

1. [CREATE TABLE in Apache Cassandra](https://docs.datastax.com/en/archived/cql/3.0/cql/cql_reference/create_table_r.html)

2. [Apache Cassandra Data Modeling and Query Best Practices](https://www.red-gate.com/simple-talk/sql/nosql-databases/apache-cassandra-data-modeling-and-query-best-practices/) 

3. [Difference between partition key, composite key and clustering key in Cassandra](https://stackoverflow.com/questions/24949676/difference-between-partition-key-composite-key-and-clustering-key-in-cassandra)

4. [Clustering columns in Apache Cassandra](https://docs.datastax.com/en/dse/5.1/cql/cql/cql_using/whereClustering.html)